In [1]:
# run the header that sets up the environment

already_run = globals().get("already_run", False)
if not (already_run):
    print("Running the header")
    already_run = True
    %run header.ipynb

    load_packages()
    change_wordir()
    colors, colours = colourblind_palette()


#model = "explicit_PSI"
#model = "QSSA_PSI"
#model = "cyclic_2021"
#model = "new_PSI"
#model = "new_PSII"
model = "latest_dev"

m = load_model(model)

Running the header


Could not find cannot import name 'dopri5' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find cannot import name 'rodas' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find cannot import name 'odassl' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find ODEPACK functions.
Could not find RADAR5
Could not find GLIMDA.


1.30.0

succesfully loaded latest_dev :D


# Model

In [2]:
stoichio_subset = m.get_stoichiometric_df()
stoichio_subset = stoichio_subset.loc[["B0", "B1", "B2", "PQ"],:]

#stoichio_subset.loc[(stoichio_subset != 0).any(axis = 1)]
stoichio_subset.loc[:, (stoichio_subset != 0).any(axis = 0)] # checks in each column, but I want to check in each row


,vB01,vB10F,vB10Q,vB12,vB20,vB23,vB32F,vB32Q,vB6f,vCyc,vNDH,vPTOX
B0,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B1,1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B2,0.0,0.0,0.0,1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0
PQ,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,1.0,-1.0,-1.0,1.0


In [3]:
#m.get_algebraic_module("P700+FA_alm")
print(m.get_algebraic_module("k_b6f"))
#m.get_algebraic_module("rel_P700+FA_alm")

k_b6f(pH, pKreg, b6f_content, max_b6f) -> ['k_b6f']


In [4]:
#m.get_compound_stoichiometry("PC")
m.get_compound_stoichiometry("PQ")

{'vB20': -0.5, 'vPTOX': 1, 'vNDH': -1, 'vB6f': 1, 'vCyc': -1}

In [5]:
#m.get_parameter("k1")
m.get_parameter("pKreg")

6.5

In [6]:
#m.get_rate_args("v2_to_P700FA-")
m.get_rate_args("vB6f")

['PC', 'PCred', 'PQ', 'PQred', 'k_b6f', 'Keq_B6f']

In [7]:
#m.get_rate_function("v2_to_P700FA-")(1,2,3,4,5,6)
m.get_rate_function("vB32Q")(1,0,1,2)

2

In [7]:
m.check_unused_compounds()

set()

In [8]:
m.check_missing_parameters()

set()

In [2]:
set(m.check_unused_parameters())- {'APX0','ATPcyt','CN','Ca','Ctot','E0_cytf','Keq_cyc','Km161','Km162','Kmnadph','NADPHcyt','bH','dT','k1','k1APX','k1rev','k2APX',
                                    'k3APX','k4','k4APX','k5','k5APX','k6','k7','k8','kATPcons','kActATPase','kDeactATPase','kNADPHcons','kNh','kNr','kProtonation','kdeg','krep','nH','pHmedium','prob_attach'}

set()

# Simulator

In [3]:
y0 = {
    "PQ": 11.027139850905353,"PC": 1.8895071932002812,"Fd": 3.8690237263896705,"ATP": 1.620195002854852,"NADPH": 0.4882103700673736,"H": 0.0022147075094596015,
    "LHC": 0.8023074419510501,"Psbs": 0.9607146039898598,"Vx": 0.950783616933656,"PGA": 0.9913970817549008,"BPGA": 0.0005355311557548053,"GAP": 0.0062630116252017295,
    "DHAP": 0.13778623933075737,"FBP": 0.006126990841013743,"F6P": 0.31166103888161867,"G6P": 0.7168203893211117,"G1P": 0.041575582577936025,"SBP": 0.01311315151803723,
    "S7P": 0.15782894767619207,"E4P": 0.00732079113061801,"X5P": 0.022396849486562384,"R5P": 0.03751472214765548,"RUBP": 0.13153657267999222,"RU5P": 0.015005888732707041,
    "MDA": 5.85270097771621e-06,"ASC": 9.999994138785292,"H2O2": 3.4273920330125316e-06,"DHA": 8.513863740903352e-09,"GSH": 9.999999991725186,"GSSG": 4.137406632226743e-09,
    "TR_ox": 0.9,"E_inactive": 4.7368421052631575,
    "P700FA": 1.506615384275408,            #eq at pfd 800       #"PSItot": 2.5, (in parameter vector of Matuszynska)
    "P700+FA-": 0.019197449388051676,
    "P700FA-": 0.028144516332212766,
    "B0": 1.9379789566530539,  # eq at pfd 800
    "B1": 9.786232812526368e-08,
    "B2": 0.5620208537555176
}

# y0 = {'P700FA': 2.5,
#     'P700pFAm': 0.0,
#     'P700FAm': 0.0,
#     'PC': 0,
#     'Fd': 5.0,
#     'ps2cs': 0.525}


In [4]:
s = Simulator(m)
s.initialise(y0)
t_end = 100
steps_per_second = 100

steps = steps_per_second * t_end

pfd = 100
s.update_parameter("pfd", pfd)

_ = s.simulate(t_end = t_end, steps = steps)

In [5]:
# s.get_full_results_df().plot(figsize = (15,10))

In [6]:
results = s.get_full_results_df()

# check if results are identical

In [9]:
#results

In [ ]:
#results2 = results.copy()

In [ ]:
joblib.dump(results, "tmp/result_test.joblib")

['tmp/result_weird_function.joblib']

In [ ]:
results2 = joblib.load("tmp/result_test.joblib")

In [7]:
results2.columns[np.where(results.columns != results2.columns)]

Index(['B3', 'PQred', 'PCred', 'Fdred', 'ADP', 'NADP', 'LHCp', 'Zx', 'Psbsp',
       'ps2cs', 'Q', 'Fluo', 'Pi', 'N'],
      dtype='object')

In [8]:
# reorder the columns if necessary!
results = results[results2.columns]

In [9]:
(results2 == results).all().all()

np.False_

In [20]:
(abs(1- results2 / results) < 1e-3 ).all().all()

np.True_

In [28]:
(1-results2 / results).mean().mean()

np.float64(-4.1035795207128936e-09)

In [13]:
abs(1- results2 / results)

,PQ,PC,Fd,ATP,NADPH,H,LHC,Psbs,Vx,PGA,...,rel_P700+FA,rel_P700FA,rel_P700FA-,rel_P700+FA-,rel_P700,rel_P700+,rel_B0,rel_B1,rel_B2,rel_B3
0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0.01,9.303669e-14,1.654232e-13,1.398881e-14,3.996803e-15,4.440892e-16,2.087219e-14,1.110223e-16,8.881784e-16,2.220446e-16,1.110223e-15,...,5.876410e-13,1.909584e-14,8.659740e-14,2.468026e-13,1.931788e-14,5.860867e-13,8.062440e-13,8.093526e-13,1.016109e-11,4.624010e-07
0.02,3.608225e-14,2.795542e-13,2.575717e-14,8.992806e-15,7.771561e-16,5.018208e-14,5.551115e-16,9.992007e-16,0.000000e+00,1.110223e-15,...,5.208944e-12,1.518785e-13,7.982504e-14,4.102718e-12,1.517675e-13,5.204281e-12,2.442491e-15,1.554312e-15,2.819966e-14,1.310776e-07
0.03,2.065015e-14,1.503464e-12,1.110223e-16,6.550316e-15,1.110223e-15,4.880540e-13,4.440892e-16,1.221245e-15,2.220446e-16,7.771561e-16,...,3.470291e-11,9.752199e-13,1.205036e-12,2.438483e-11,9.758860e-13,3.465761e-11,4.773959e-15,5.218048e-15,6.394885e-14,8.771773e-08
0.04,2.613465e-13,4.012124e-12,3.996803e-15,5.362377e-14,3.996803e-15,1.104006e-12,5.551115e-16,1.332268e-15,2.220446e-16,2.553513e-15,...,2.583378e-11,7.189804e-13,8.397727e-13,1.858091e-11,7.192025e-13,2.580203e-11,5.129230e-14,5.162537e-14,6.596945e-13,1.097173e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.96,6.049983e-09,3.067209e-09,1.299816e-09,6.526417e-09,3.229557e-09,1.448655e-09,4.092306e-10,8.811986e-09,1.320919e-09,2.009907e-08,...,3.799813e-09,7.265410e-11,4.914026e-09,1.639074e-09,8.846035e-11,3.770230e-09,1.495591e-09,3.694397e-10,1.332976e-08,8.664480e-08
99.97,6.061073e-09,3.082211e-09,1.306336e-09,6.597781e-09,3.241821e-09,1.631887e-09,4.090401e-10,8.811794e-09,1.320942e-09,2.029647e-08,...,3.836454e-09,7.341705e-11,4.943476e-09,1.627542e-09,8.931711e-11,3.806735e-09,1.498744e-09,3.660420e-10,1.335796e-08,3.938429e-08
99.98,6.072249e-09,3.097434e-09,1.312899e-09,6.669077e-09,3.254082e-09,1.817227e-09,4.088501e-10,8.811601e-09,1.320966e-09,2.049369e-08,...,3.873631e-09,7.419232e-11,4.973162e-09,1.615658e-09,9.018664e-11,3.843774e-09,1.501904e-09,3.626324e-10,1.338623e-08,7.876916e-09
99.99,6.083511e-09,3.112880e-09,1.319504e-09,6.740306e-09,3.266339e-09,2.004689e-09,4.086600e-10,8.811406e-09,1.320989e-09,2.069072e-08,...,3.911374e-09,7.497980e-11,5.003087e-09,1.603420e-09,9.106915e-11,3.881379e-09,1.505074e-09,3.592107e-10,1.341459e-08,3.938487e-08
